In [15]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

### Carga data

In [12]:
infile = open('cleandata_pickle.pickle','rb')
data_clean = pickle.load(infile)
infile.close()

### Gráficas

In [ ]:
f = plt.subplots(figsize = (6,7))

sns.scatterplot(data = data_clean[data_clean['año_cierre'] == 2019] , y = "latitud", x = "longitud", hue = "delegacion_cierre")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)